In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
test_path = '/content/drive/My Drive/circuits/c499.test'

In [4]:
import itertools
import random
from collections import defaultdict
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

def expand_dont_care_limited(pattern, max_expand=15):
    x_indices = [i for i, c in enumerate(pattern) if c == 'x']
    if len(x_indices) <= max_expand:
        chars = [(c if c in '01' else ['0', '1']) for c in pattern]
        chars = [(c if isinstance(c, list) else [c]) for c in chars]
        return [''.join(bits) for bits in itertools.product(*chars)]
    else:
        fixed_pattern = list(pattern)
        patterns = []
        for bits in itertools.product('01', repeat=max_expand):
            temp = fixed_pattern.copy()
            for i, bit in zip(x_indices[:max_expand], bits):
                temp[i] = bit
            for i in x_indices[max_expand:]:
                temp[i] = random.choice('01')
            patterns.append(''.join(temp))
        return patterns

def parse_isc_file_limited(filepath, max_expand=3):
    combo_to_fault = {}
    fault_to_idx = {}
    fault_type_map = {}
    idx = 0
    current_fault = None

    with open(filepath, 'r') as file:
        for line in file:
            line = line.strip()
            if not line or line.startswith("*"):
                continue
            if '/' in line:
                parts = line.split("/")
                fault = parts[0].strip().replace("->", "_") + "/" + parts[1].strip()
                if fault not in fault_to_idx:
                    fault_to_idx[fault] = idx
                    if parts[1].strip() == "0":
                        fault_type_map[idx] = "stuck-at-0"
                    elif parts[1].strip() == "1":
                        fault_type_map[idx] = "stuck-at-1"
                    else:
                        fault_type_map[idx] = "unknown"
                    idx += 1
                current_fault = fault
            elif ':' in line and current_fault:
                parts = line.split(":")[1].strip().split()
                input_pat = parts[0]
                output_pat = parts[1] if len(parts) > 1 else ""
                for xi in expand_dont_care_limited(input_pat, max_expand):
                    for yo in expand_dont_care_limited(output_pat, max_expand):
                        combined = xi + yo
                        if combined not in combo_to_fault:
                            combo_to_fault[combined] = fault_to_idx[current_fault]
    return combo_to_fault, fault_to_idx, fault_type_map

In [5]:
combo_to_fault, fault_to_idx, fault_type_map = parse_isc_file_limited(test_path, max_expand=3)

X = []
y = []

for combo, label in combo_to_fault.items():
    X.append([int(bit) for bit in combo])
    y.append(label)

X = np.array(X)
y = np.array(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [6]:
svm_model = SVC(kernel='linear', C=1.0, decision_function_shape='ovr')
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)

type_correct = defaultdict(int)
type_total = defaultdict(int)

for true, pred in zip(y_val, y_pred):
    ftype = fault_type_map[true]
    type_total[ftype] += 1
    if true == pred:
        type_correct[ftype] += 1

In [7]:
print(f"\n--- Final Overall Validation Accuracy ---")
print(f"Accuracy = {accuracy:.4f} ({(accuracy * len(y_val)):.0f}/{len(y_val)})")

print("\n--- Final Fault-Type Accuracy ---")
for ftype in sorted(type_total):
    acc = type_correct[ftype] / type_total[ftype]
    print(f"{ftype}: Accuracy = {acc:.4f} ({type_correct[ftype]}/{type_total[ftype]})")


--- Final Overall Validation Accuracy ---
Accuracy = 0.9548 (13866/14522)

--- Final Fault-Type Accuracy ---
stuck-at-0: Accuracy = 0.9639 (6033/6259)
stuck-at-1: Accuracy = 0.9480 (7833/8263)
